In [ ]:
%%html
<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [3]'.</span>

An Exception was encountered at 'In [3]'.

In [1]:
import os 
import sys
import pandas as pd
import time
import bs4
from bs4 import BeautifulSoup
import requests
import re
import json
import datetime
import sqlite3
import logging
%load_ext autoreload
%autoreload 2

In [2]:
current_dt = datetime.datetime.now()
current_date = str(current_dt.date())
current_time = str(current_dt.time().strftime('%H:%M:%S'))
print(current_dt)

2020-03-01 21:53:10.074519


In [3]:
link_homepage = 'https://meh.com'
page = requests.get(link_homepage)
print('Status Code', page.status_code)
soup = BeautifulSoup(page.content, 'html.parser')

ConnectionError: HTTPSConnectionPool(host='meh.com', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x70025bd0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

## Save Raw Response

In [ ]:
# os.chdir('..')
db_location = '../data/meh_scraper_qa.db'

In [ ]:
# Parameters
db_location = "data/meh_scraper.db"


In [ ]:
con = sqlite3.connect(db_location)
cursor = con.cursor()

In [ ]:
stats = soup.find(class_='community-stats')

In [ ]:
stats1 = str(stats)
stats2 = stats1.replace('\'','\'\'')

In [ ]:
cursor.execute("""INSERT INTO raw_site_community_stats (date, time, raw_site) 
        VALUES ('{0}', '{1}', '{2}')""".format(current_date, current_time, stats2))

In [ ]:
con.commit()

In [ ]:
con.close()

## Parse Wanted Fields

In [ ]:
parsed_nums = {}
parsed_nums['date'] = current_date
parsed_nums['time'] = current_time

### Page Views

In [ ]:
page_views = stats.find('div', {'id':'pageviews'})

In [ ]:
page_view_secondary = page_views.find(class_='secondary').get_text()
secondary_parts = page_view_secondary.rstrip().lstrip().split(', ')
for visit_method, method_num in zip(secondary_parts, range(1,3)):
    visit_parts = visit_method.split('% ')
    parsed_nums['Page Views ' + str(method_num)+' Name'] = visit_parts[1]
    parsed_nums['Page Views ' + str(method_num)+' Percent'] = int(visit_parts[0])

In [ ]:
primary_class = page_views.find_all(class_='primary')
primary_numbs = [x.find('strong').get_text() for x in primary_class]
parsed_nums['# Visited'] = int(primary_numbs[0])
parsed_nums['Clicked Meh'] = int(primary_numbs[1])


In [ ]:
parsed_nums

### Referrals

In [ ]:
referrals = stats.find('div', {'id':'referrals'})

In [ ]:
parsed_nums['Typed Meh Percent'] = int(referrals.find(class_='primary').find('strong').get_text()[:-1])

In [ ]:
parsed_nums['Referrals Percent'] = int(referrals.find(class_='secondary').find('strong').get_text()[:-1])

In [ ]:
referrers = referrals.find_all(class_='referrer')
for ref, ref_num in zip(referrers, range(1, len(referrers)+1)):
    parsed_nums['Referrer '+str(ref_num) + ' Percent'] = float(ref['data-percentage'])
    parsed_nums['Referrer '+str(ref_num) + ' Name'] = ref.get_text().lstrip().rstrip().split(' \n')[0]
    

### Order Counts

In [ ]:
order_stats = stats.find('div', {'id': 'order-stats'})

In [ ]:
order_nums = [re.findall('\d+', x.get_text())[0] for x in order_stats.find_all(class_='primary')]

In [ ]:
parsed_nums['Items Sold'] = order_nums[0]
parsed_nums['Dollars Sold'] = order_nums[1]

## Export

In [ ]:
export_series = pd.DataFrame(parsed_nums, index=[1])

In [ ]:
export_series

In [ ]:
con = sqlite3.connect(db_location)
cursor = con.cursor()
export_series.to_sql('selling_details', con, if_exists='append', index=False)

In [ ]:
con.commit()
con.close()